## Data cleaning
# Two datasets: International Soil Moisture Network (ISMN) and Estonian Environment Agency's historical weather data.

**ISMN data cleaning**

In [ ]:
###script to group files to folders based on the availability of all variables
###example:
#folder_with_p_and_ta
#folder_with_p_and_ta_and_sm

import os
import shutil


base_dir = r"/path/to/your/folder"
output_dir = r"/path/to/your/folder"
os.makedirs(output_dir, exist_ok=True)
keywords = {
    "_sm_": "sm",
    "_ta_": "ta",
    "_p_": "p"}

def extract_keywords(filename, keywords):
    name_lower = filename.lower()
    return [kw for kw in keywords if kw.lower() in name_lower]

for root, dirs, files in os.walk(base_dir):

    if os.path.abspath(root) == os.path.abspath(output_dir):
        continue

    # Find CSV files
    stm_files = [f for f in files if f.lower().endswith(".stm")]

    if not stm_files:
        continue

    # Collect keywords found inside this folder
    detected_keywords = set()

    for file in stm_files:
        found = extract_keywords(file, keywords)
        detected_keywords.update(found)

    # If no keywords were found at all, skip
    if not detected_keywords:
        continue

    # Sort keywords to create consistent folder names
    keywords_sorted = sorted(detected_keywords)

    # Build folder name
    folder_name = "Folder_with_" + "_and_".join(keywords_sorted)

    # Create the output folder
    new_folder_path = os.path.join(output_dir, folder_name)
    os.makedirs(new_folder_path, exist_ok=True)

    # Copy matching files
    for stm in stm_files:
        src = os.path.join(root, stm)
        dst = os.path.join(new_folder_path, stm)
        shutil.copy2(src, dst)

    print(f"Created: {new_folder_path}")




In [ ]:
###STM_TO_CSV convert####

import os
import pandas as pd
from tqdm import tqdm

def process_stm_file(filepath, filename):
    """
    Reads a .stm file, extracts metadata (first line), and processes the remaining content
    into a DataFrame with predefined headers.
    """
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    if not lines:
        return None, None  # Return None if file is empty

    metadata = lines[0].strip()

    for line in lines[1:]:  # Skip the first line (metadata)
        line = line.strip()
        if not line:
            continue  # Skip empty lines
        row = [cell for cell in line.split(" ") if cell != ""]  # Split on spaces, remove empty values
        data.append(row)

    df = pd.DataFrame(data)
    if "_sm_" in filename:
        value_header = "value (m3*m3)"
    elif "_ta_" in filename:
        value_header = "value (C)"
    elif "_p_" in filename:
        value_header = "value (mm)"
    else:
        raise ValueError(f"Unknown file in {filename}")


    headers = ["date", "time", value_header, "quality_flag", "source_flag"]

    while df.shape[1] < len(headers):
        df[df.shape[1]] = None  # Fill missing columns with NaN

    df.columns = headers
    return df, metadata

def count_total_stm_files(root_folder):
    """Counts the total number of .stm files across all subfolders in the root folder."""
    total_files = 0
    for dirpath, _, filenames in os.walk(root_folder):
        total_files += len([f for f in filenames if f.endswith(".stm")])
    return total_files

def convert_stm_folders(root_folder, output_folder):
    """
    Processes all .stm files in the root folder and its subfolders, converting them to csv format.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    total_files = count_total_stm_files(root_folder)

    with tqdm(total=total_files, desc="Converting .stm files") as pbar:
        for dirpath, _, filenames in os.walk(root_folder):
            relative_path = os.path.relpath(dirpath, root_folder)
            subfolder_output_path = os.path.join(output_folder, relative_path)
            if not os.path.exists(subfolder_output_path):
                os.makedirs(subfolder_output_path)

            for filename in filenames:
                if filename.endswith(".stm"):
                    stm_path = os.path.join(dirpath, filename)
                    output_csv = os.path.join(subfolder_output_path, f"{os.path.splitext(filename)[0]}.csv")

                    df, metadata = process_stm_file(stm_path, filename)
                    if df is None:
                        print(f"Skipping empty file: {filename}")
                        pbar.update(1)
                        continue

                                        # Save main data to CSV
                    df.to_csv(output_csv, index=False)

                    # Save metadata to a separate CSV if it exists
                    if metadata:
                        metadata_filename = f"{os.path.splitext(filename)[0]}_metadata.csv"
                        metadata_path = os.path.join(metadata_output_folder, metadata_filename)
                        metadata_df = pd.DataFrame([{"Metadata": metadata}])
                        metadata_df.to_csv(metadata_path, index=False, header=False)

                    pbar.update(1)

# Define root input folder
root_folder = r"/path/to/your/folder"
output_folder = r"/path/to/your/folder"

metadata_output_folder = os.path.join(root_folder, "metadata_files")
os.makedirs(metadata_output_folder, exist_ok=True)

convert_stm_folders(root_folder, output_folder)
print("Conversion complete!")



In [ ]:
### Cleaning prodeuced csv and calculating daily average
### If dubious or missing values, mean will be calculated from +-1 values
### only values with quality flag "G" --> good, accepted


import pandas as pd
import os
from tqdm import tqdm

csv_folder = r"/path/to/your/folder"
output_folder = r"/path/to/your/folder"
os.makedirs(output_folder, exist_ok=True)

files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]
#COSMOS-UK_COSMOS-UK_AliceHolt_sm_0.000000_0.300000_Cosmic-ray-Probe_20100101_20251105
for filename in tqdm(files, desc="Processing csv fikes"):
    filepath = os.path.join(csv_folder, filename)
    df = pd.read_csv(filepath)
    pices = filename.split("_")
    network = pices[1]
    station = pices[2]
    depth_from = pices[4]
    depth_to = pices[5]
    sensor = pices[6]

    # Determine value column name based on filename
    if "_sm_" in filename:
        value_col = "value (m3*m3)"
    elif "_ta_" in filename:
        value_col = "value (C)"
    elif "_p_" in filename:
        value_col = "value (mm)"
    else:
        print(f"Unknown value column for file: {filename}")
        continue

    # Replace -9999 with NaN
    df[value_col] = pd.to_numeric(df[value_col], errors='coerce')
    df[value_col] = df[value_col].replace(-9999, pd.NA)

    # Combine date and time into datetime index
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], errors='coerce')
    df = df.dropna(subset=['datetime'])  # Drop rows with invalid datetime
    df.set_index('datetime', inplace=True)

    # Filter good quality values
    df_good = df[df['quality_flag'] == 'G']

    # Daily mean from good values
    daily_good = df_good.resample('D')[value_col].mean()

    # All days in original data
    all_days = df.resample('D')[value_col].mean()
    final_daily = []

    for date in all_days.index:
        if pd.isna(daily_good.get(date)):
            # Fallback to ±1 month average
            start = date - pd.DateOffset(days=30)
            end = date + pd.DateOffset(days=30)
            window = df[(df.index >= start) & (df.index <= end)]
            fallback_mean = window[value_col].mean()
            rounded_value = round(fallback_mean, 4)
            final_daily.append((date.date(), rounded_value, "fallback"))
        else:
            rounded_value = round(daily_good[date], 4)
            final_daily.append((date.date(), rounded_value, "good"))


    # Save result
    result_df = pd.DataFrame(final_daily, columns=['date', 'daily_avg', 'source'])
    unit = value_col.split("(")[-1].strip(")")
    result_df.rename(columns={value_col: f'daily_avg ({unit})'}, inplace=True)

    ### adding network, station, depth and sensor data from filename
    result_df["network"] = network
    result_df["station"] = station
    result_df["depth_from"] = depth_from
    result_df["depth_to"] = depth_to
    result_df["sensor"] = sensor
    output_path = os.path.join(output_folder, filename)
    result_df.to_csv(output_path, index=False)

print("All csv cleaned")


In [ ]:

### CSV merge to precipitation, soilmoisture and airtemperature csv-s

import os
import pandas as pd
from tqdm import tqdm

def merge_csv_files(input_folder, output_folder):
    # Define the patterns to search for in filenames
    patterns = ["_sm_", "_p_", "_ta_", "_ts_"]

    # Create a dictionary to hold DataFrames for each pattern
    dfs = {pattern: [] for pattern in patterns}

    # Get the list of all CSV files in the input folder and subfolders
    csv_files = [os.path.join(root, file)
                 for root, _, files in os.walk(input_folder)
                 for file in files if file.endswith(".csv")]

    # Iterate over files with a progress bar
    for file_path in tqdm(csv_files, desc="Merging CSV files"):
        try:
            # Read the CSV file
            df = pd.read_csv(file_path, encoding='utf-8')

            # Check which pattern the file matches and add to the corresponding list
            for pattern in patterns:
                if pattern in file_path:
                    dfs[pattern].append(df)
                    break
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")

    # Merge DataFrames for each pattern and save to separate CSV files
    for pattern, df_list in dfs.items():
        if df_list:
            merged_df = pd.concat(df_list, ignore_index=True)
            output_file = os.path.join(output_folder, f"merged_{pattern.strip('_')}.csv")
            try:
                merged_df.to_csv(output_file, index=False, encoding='utf-8')
                print(f"Merged {len(df_list)} files into {output_file}")
            except Exception as e:
                print(f"Error writing file {output_file}: {e}")

# Define input and output folders
input_folder = r"/path/to/your/folder"
output_folder = r"/path/to/your/folder"
os.makedirs(output_folder, exist_ok=True)

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Merge CSV files
merge_csv_files(input_folder, output_folder)
print("Merging complete!")

In [ ]:
### Creating inner join between previous sm, ta and p csv-s
### Creating date filter

import pandas as pd

df_sm = pd.read_csv("/path/to/your/file/merged_sm.csv")
df_ta = pd.read_csv("/path/to/your/file/merged_ta.csv")
df_p = pd.read_csv("/path/to/your/file/merged_p.csv")

df_sm.reset_index(drop=True, inplace=True)
df_ta.reset_index(drop=True, inplace=True)
df_p.reset_index(drop=True, inplace=True)


df_sm.rename(columns={"daily_avg": "soil_moisture",
                      "depth_from": "sm_depth_from",
                      "depth_to": "sm_depth_to",
                      "source": "sm_source",}, inplace=True)
df_ta.rename(columns={"daily_avg": "temperature",
                      "source": "ta_source",
                      "depth_from": "ta_depth_from",
                      "depth_to": "ta_depth_to",
                      "sensor": "ta_sensor"}, inplace=True)
df_p.rename(columns={"daily_avg": "precipitation",
                     "source": "p_source",
                      "depth_from": "p_depth_from",
                      "depth_to": "p_depth_to",
                      "sensor": "p_sensor"}, inplace=True)

df_sm.drop(["network"], axis=1, inplace=True)
df_ta.drop(["network"], axis=1, inplace=True)


merged = pd.merge(df_sm, df_ta, on=["station", "date"], how="inner")
merged = pd.merge(merged, df_p, on=["station", "date"], how="inner")

output = merged[merged["date"] >= "2018-01-01"]


output.to_csv("/path/to/your/file/ISMN_data_clean.csv", index=False)

In [ ]:
### ISMN sanity

In [ ]:
import pandas as pd
ismn_data = pd.read_csv("/path/to/your/file/ISMN_data_clean.csv")
ismn_data.head()
ismn_data["network"].unique()
ismn_data.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ismn_data['date'] = pd.to_datetime(ismn_data['date'])

#Checking different stations or networks
year1 = ismn_data[(ismn_data["date"] < "2019-01-01") & (ismn_data["network"] == "RSMN")]

plt.figure(figsize=(12, 6))
sns.lineplot(data=year1, x="date", y="soil_moisture", label="Soilmoisture")
sns.lineplot(data=year1, x="date", y="precipitation", label="precipitation")
sns.lineplot(data=year1, x="date", y="temperature", label="Air temperature")
plt.xlabel("Year")
plt.ylabel("SM, TA, P values")
plt.title("ISMN data cleaned")
plt.legend()
plt.show()

Estonian data cleaning

In [ ]:
## Cleaning Estonian data
## Kert

import pandas as pd

df = pd.read_csv("Station_Name.csv",
                 low_memory=False,
                 encoding="utf-8",
                 sep=";",
                 header=1)

df = df[[
    "Aasta",
    "Kuu",
    "Päev",
    "Tunni sademete summa mm",
    "Õhutemperatuur °C"
]]

df['Õhutemperatuur °C'] = df['Õhutemperatuur °C'].str.replace(',', '.').astype(float)
df['Tunni sademete summa mm'] = df['Tunni sademete summa mm'].str.replace(',', '.').astype(float)

first_valid_index = df['Tunni sademete summa mm'].first_valid_index()
df = df.loc[first_valid_index:]

# Excluse empty and NaN values
df = df[df['Tunni sademete summa mm'].notna() & (df['Tunni sademete summa mm'] > 0)]

# Daily average
df_daily = df.groupby(['Aasta','Kuu','Päev'], as_index=False).agg({
    'Tunni sademete summa mm':'mean',
    'Õhutemperatuur °C':'mean'
})

# Transform dates to match ISMN data structure
df_daily['Kuupäev'] = pd.to_datetime(
    df_daily['Aasta'].astype(str) + '-' +
    df_daily['Kuu'].astype(str) + '-' +
    df_daily['Päev'].astype(str),
    format='%Y-%m-%d'
)

df_daily_final = df_daily[['Kuupäev','Tunni sademete summa mm','Õhutemperatuur °C']]

df_daily_final.to_csv("Station_Name_clean.csv", index=False, sep=',', float_format='%.3f', encoding='utf-8-sig')

print(df_daily_final.head())

# Later found that forgot to change the attribute names (column names) to english language - this was performed on the main script.

In [ ]:
###